In [3]:
import pandas as pd
from transformers import pipeline

# Load the dataset
csv_file_path = r'C:\Users\bhavi\OneDrive\Documents\Conestoga courses\AI Sem1\ML\sentiment-analysis-system\New Data-collection\reddit.csv'
df = pd.read_csv(csv_file_path)  # Load the data into a DataFrame

# Load the sentiment analysis model
classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

# Define a function to annotate sentiment
def annotate_sentiment(text):
    try:
        result = classifier(text)
        return result[0]['label'].lower()
    except Exception as e:
        print(f"Error processing text: {e}")
        return 'neutral'

# Annotate each post with a sentiment label
df['Sentiment'] = df['Title'].apply(annotate_sentiment)

# Save the annotated dataset to a new CSV file
annotated_file_path = r'C:\Users\bhavi\OneDrive\Documents\Conestoga courses\AI Sem1\ML\sentiment-analysis-system\New Data-collection\Labelled.csv'
df.to_csv(annotated_file_path, index=False)

print(f"Annotated dataset saved to {annotated_file_path}")


All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


Annotated dataset saved to C:\Users\bhavi\OneDrive\Documents\Conestoga courses\AI Sem1\ML\sentiment-analysis-system\New Data-collection\Labelled.csv


In [5]:
import pandas as pd
from transformers import pipeline

# Load the dataset
csv_file_path = r'C:\Users\bhavi\OneDrive\Documents\Conestoga courses\AI Sem1\ML\sentiment-analysis-system\New Data-collection\reddit.csv'
df = pd.read_csv(csv_file_path)  # Load the data into a DataFrame

# Load the sentiment analysis model
classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

# Updated function to annotate sentiment including 'neutral'
def annotate_sentiment(text):
    try:
        # Get the result from the classifier
        result = classifier(text)
        # Get the label and score
        label = result[0]['label'].lower()
        score = result[0]['score']

        # Set thresholds for determining 'neutral' sentiment
        neutral_threshold_upper = 0.6
        neutral_threshold_lower = 0.4

        # If the score is not confident, label as 'neutral'
        if neutral_threshold_lower <= score <= neutral_threshold_upper:
            return 'neutral'
        else:
            return label
    except Exception as e:
        print(f"Error processing text: {e}")
        return 'neutral'

# Annotate each post with a sentiment label
df['Sentiment'] = df['Title'].apply(annotate_sentiment)

# Save the annotated dataset to a new CSV file
annotated_file_path = r'C:\Users\bhavi\OneDrive\Documents\Conestoga courses\AI Sem1\ML\sentiment-analysis-system\New Data-collection\Labelled.csv'
df.to_csv(annotated_file_path, index=False)

print(f"Annotated dataset saved to {annotated_file_path}")


All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


Annotated dataset saved to C:\Users\bhavi\OneDrive\Documents\Conestoga courses\AI Sem1\ML\sentiment-analysis-system\New Data-collection\Labelled.csv


In [6]:
import pandas as pd

# Load the annotated CSV file
df = pd.read_csv('C:\\Users\\bhavi\\OneDrive\\Documents\\Conestoga courses\\AI Sem1\\ML\\sentiment-analysis-system\\New Data-collection\\Labelled.csv')

# Count the number of occurrences of each sentiment label
sentiment_counts = df['Sentiment'].value_counts()

# Print the count of each sentiment
print(sentiment_counts)


Sentiment
negative    3281
positive    2216
neutral      152
Name: count, dtype: int64


In [9]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from emoji import demojize
import re

# NLTK Downloads
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load the dataset
df = pd.read_csv('C:\\Users\\bhavi\\OneDrive\\Documents\\Conestoga courses\\AI Sem1\\ML\\sentiment-analysis-system\\New Data-collection\\Labelled.csv')

# Slang dictionary
slang_dict = {
    'btw': 'by the way',
    'lol': 'laughing out loud',
    'idk': 'I do not know',
    'imo': 'in my opinion',
    'imho': 'in my humble opinion',
    'brb': 'be right back',
    'tbh': 'to be honest',
    'lmao': 'laughing my ass off',
    'rofl': 'rolling on the floor laughing',
    'smh': 'shaking my head',
    'omg': 'oh my god',
    'ttyl': 'talk to you later',
    'afaik': 'as far as I know',
    'irl': 'in real life',
    'thx': 'thanks',
    'pls': 'please',
    'dm': 'direct message',
    'fyi': 'for your information',
    'b4': 'before',
    'gr8': 'great',
    'u': 'you',
    'r': 'are',
    'yolo': 'you only live once',
    'np': 'no problem',
    'g2g': 'got to go',
    'tldr': 'too long, didn’t read',
    'jk': 'just kidding',
    'bff': 'best friends forever',
    'icymi': 'in case you missed it',
    'fomo': 'fear of missing out',
    'ftw': 'for the win',
    'wtf': 'what the f***',
    'nsfw': 'not safe for work',
    'nbd': 'no big deal',
    'faq': 'frequently asked questions',
    'afk': 'away from keyboard',
    'asap': 'as soon as possible'
}

# Function to extract and process hashtags
def process_hashtags(text):
    hashtags = re.findall(r'#\w+', text)
    return ' '.join(hashtags)

# Preprocessing function
def preprocess_text(text):
    # Convert emojis to text and process hashtags
    text = demojize(text) + ' ' + process_hashtags(text)

    # Tokenize
    tokens = nltk.word_tokenize(text)

    # Remove stopwords and apply lemmatization
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens if token.lower() not in stop_words]

    # Translate slang
    tokens = [slang_dict.get(token, token) for token in tokens]

    # Remove non-alphabetic characters and keep the tokens
    tokens = [token for token in tokens if token.isalpha()]

    return ' '.join(tokens)

# Apply preprocessing to each title
df['Processed_Title'] = df['Title'].apply(preprocess_text)

# Define the file path to save the preprocessed dataset
preprocessed_file_path = 'C:\\Users\\bhavi\\OneDrive\\Documents\\Conestoga courses\\AI Sem1\\ML\\sentiment-analysis-system\\New Data-collection\\Preprocessed.csv'

# Save the preprocessed dataset
df.to_csv(preprocessed_file_path, index=False)

print(f"Preprocessed dataset saved to {preprocessed_file_path}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bhavi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bhavi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\bhavi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Preprocessed dataset saved to C:\Users\bhavi\OneDrive\Documents\Conestoga courses\AI Sem1\ML\sentiment-analysis-system\New Data-collection\Preprocessed.csv
